In [1]:
import pandas    as pd
import datetime as dt
import numpy as np
import pickle

### train data

In [2]:
data = pd.read_csv(r'G:\DATASETS\Analytics Vidya\train_8wry4cB.csv')
#converting object data field into date formate
data['start'] = data['startTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
data['end'] = data['endTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
## duration time in the time formate
data['session_time'] = data['end']-data['start']
# function for changing duration values to minutes int type
def change_to_minutes(st):
    data_list = str(st).split(':')
    return int(data_list[1])+int(data_list[2])/60
data['duration'] = data['session_time'].apply(change_to_minutes)
#dropping extra columns 
data.drop(data.iloc[:,[0,1,2,5,6,7]],axis=1,inplace=True)

products = []   # set of products 
ind_prds = []   # all product names
for dt in data['ProductList']:
    lst = dt.split(';')
    
    products.extend(lst)
    for lst2 in lst:
        sd = lst2.split('/')
        ind_prds.extend(sd)
    #print(dt)
for ls in ind_prds:
    if ls=='':
        ind_prds.remove('')   
#dependent variable       
y= data['gender']
# find the most repated products 
word_dict = {}
for word in ind_prds:
    if word in word_dict:
        word_dict[word] +=1
    else:
        word_dict[word] = 1
#selecting 3750 most repeated products.     
word_final = dict(sorted(word_dict.items(),key=lambda x:x[1],reverse=True)[:3750])
# giving ranking to the selected products from the above
vocab = {}
i=0 
for word in word_final.keys():
    if word in vocab:
        continue
    else:
        vocab[word] = i
        i +=1
#train data is converting to word vectors by splitting the products in each line
total_list = []
for dt in data['ProductList']:
    lst = dt.split(';')
    inner_list = []
    for lst2 in lst:
        sd = lst2.split('/')
        inner_list.extend(sd)
    total_list.append(inner_list)
#training vector ...word vector to binary vector
train_vector = []
for line in total_list:
    lexicon2 = [0]*len(vocab.keys())
    for word in line:
        if word in vocab:
            lexicon2[vocab[word]] =1
    train_vector.append(lexicon2)
# appending duration column to correspodng binary vector
for w,d in zip(train_vector,data['duration']):
    w.append(d)    

In [14]:
len(train_vector)

10500

### test data

In [3]:
test_data = pd.read_csv(r'G:\DATASETS\Analytics Vidya\test_Yix80N0.csv')
import datetime as dt
test_data['start'] = test_data['startTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
test_data['end'] = test_data['endTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
test_data['session_time'] = test_data['end']-test_data['start']
def change_to_minutes(st):
    data_list = str(st).split(':')
    return int(data_list[1])+int(data_list[2])/60
test_data['duration'] = test_data['session_time'].apply(change_to_minutes)
ID = test_data['session_id']
test_data.drop(test_data.iloc[:,[0,1,2,4,5,6]],axis=1,inplace=True)

test_list = []
for dt in test_data['ProductList']:
    lst = dt.split(';')
    inner_list = []
    for lst2 in lst:
        sd = lst2.split('/')
        inner_list.extend(sd)
    test_list.append(inner_list)
    
test_vector = []
for line in test_list:
    lexicon2 = [0]*len(vocab.keys())
    for word in line:
        
        if word in vocab:
            #print(vocab[word])
            
            lexicon2[vocab[word]] =1
            
    #print(lexicon2)
    test_vector.append(lexicon2)

for w,d in zip(test_vector,test_data['duration']):
    w.append(d)

In [15]:
len(test_vector)

4500

In [5]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
votingom sklearn.metrics import accuracy_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

logis = LogisticRegression()
logis.fit(train_vector,y)
predictions_logis = logis.predict(train_vector)
print(f'{"LogisticRegression Accuracy ":<{40}} {accuracy_score(predictions_logis,y)}')

SGD = SGDClassifier()
SGD.fit(train_vector,y)
predictions_SGD = SGD.predict(train_vector)
print(f'{"SGDClassifier Accuracy ":<{40}} {accuracy_score(predictions_SGD,y)}')

LSVC = LinearSVC()
LSVC.fit(train_vector,y)
predictions_LSVC = LSVC.predict(train_vector)
print(f'{"LinearSVC Accuracy ":<{40}} {accuracy_score(predictions_LSVC,y)}')

RFC = RandomForestClassifier()
RFC.fit(train_vector,y)
predictions_RFC = RFC.predict(train_vector)
print(f'{"RandomForestClassifier Accuracy ":<{40}} {accuracy_score(predictions_RFC,y)}')

bagging = BaggingClassifier()
bagging.fit(train_vector,y)
predictions_bagging = bagging.predict(train_vector)
print(f'{"BaggingClassifier Accuracy ":<{40}} {accuracy_score(predictions_bagging,y)}')


LogisticRegression Accuracy              0.8939047619047619
SGDClassifier Accuracy                   0.9082857142857143
LinearSVC Accuracy                       0.9237142857142857
RandomForestClassifier Accuracy          0.947047619047619


In [16]:
predictions_bagging = bagging.predict(test_vector)
predictions_RFC = RFC.predict(test_vector)
predictions_LSVC = LSVC.predict(test_vector)
predictions_logis =logis.predict(test_vector)
predictions_SGD =SGD.predict(test_vector)


predict_data = pd.DataFrame({'predictions_bagging':predictions_bagging,
                             'predictions_LSVC':predictions_LSVC,'predictions_SGD':predictions_SGD,'predictions_logis':predictions_logis,
                            'predictions_RFC':predictions_RFC})
from statistics import mode
result = []
for i in range(len(predict_data)):
    result.append(mode(predict_data.iloc[i]))
    
filename = r'submission_vote.csv'
submission = pd.DataFrame({'session_id':ID,'gender':result})
submission.to_csv(filename,index=False)

In [17]:
submission.head()

,session_id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female
